<h2>Manually inspect images, define ROIs with labels and extract nuclei numbers positive for a cell marker within each label (3D stack to 2D MIP)</h2>

In [29]:
from pathlib import Path
import czifile
import nd2
import napari
import os
import numpy as np
import pandas as pd
from utils import segment_nuclei_2d, segment_marker_positive_nuclei

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [30]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Create an empty list to store all image filepaths within the dataset directory
images = []

# Create an empty list to store all stats extracted from each image
stats = []

# Iterate through the .czi and .nd2 files in the directory
for file_path in directory_path.glob("*.czi"):
    images.append(str(file_path))
    
for file_path in directory_path.glob("*.nd2"):
    images.append(str(file_path))
    
images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>

In [31]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[1]

# Read path storing raw image and extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file (either .czi or .nd2)
try:
    img = czifile.imread(image)
    # Remove singleton dimensions
    img = img.squeeze()
    # Perform MIP on all channels
    img_mip = np.max(img, axis=1)

except ValueError:
    img = nd2.imread(image)
    # Perform MIP on all channels
    img_mip = np.max(img, axis=0)

# Show image in Napari to define ROI
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

# Feedback for researcher
print(f"Image displayed: {filename}")
print(f"Array shape: {img.shape}")
print(f"MIP Array shape: {img_mip.shape}")

Image displayed: HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Array shape: (4, 12, 3798, 2877)
MIP Array shape: (4, 3798, 2877)


<h3>Label your regions of interest in Napari and explore the signal of your marker of interest</h3>

Make sure to set <code>n edit dim = 3</code> so the label propagates across all channels. Name your regions of interest as either <code>DG</code>, <code>CA1</code>, <code>CA3</code> or <code>HIPPO</code>. Select the <code>img_mip</code> layer and hover each pixel to later on set a threshold above which cells will be considered positive for said marker.

<video controls>
  <source src="./assets/napari_labels.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [32]:
# Code snippet to analyze cropped regions (ROIs) defined by labels or the full image if no ROI is drawn in Napari

# Initialize empty list to store the label name and Numpy arrays so we can loop across the different ROIs
layer_names = []
layer_labels = []

if len(viewer.layers) == 1:

    # Extract the xy dimensions of the input image
    img_shape = viewer.layers[0].data.shape
    img_xy_dims = img.shape[-2:]

    # Create a label covering the entire image
    label = np.ones(img_xy_dims)

    # Add a name and the label to its corresponding list
    layer_names.append("full_image")
    layer_labels.append(label)

else:

    for layer in viewer.layers:

        # Extract the label names
        label_name = layer.name
        # Ignore img_mip since it is not a user defined label
        if label_name == "img_mip":
            pass
        else:
            # Store label names
            layer_names.append(label_name)
            # Get the label data as a NumPy array to mask the image
            label = layer.data 
            layer_labels.append(label)

# Print the defined ROIs that will be analyzed
print(f"The following labels will be analyzed: {layer_names}")


The following labels will be analyzed: ['full_image']


In [ ]:
#TODO: Save user-defined ROIs in a ROI folder under directory_path/ROI as .tiff files
# Subfolders for each user-defined label region
# Store using the same filename as the input image to make things easier

<h3>Define your nuclei/marker stacks and your marker/erosion thresholds</h3>

Modify the values for <code>nuclei_channel</code>, <code>marker_channel</code>, <code>nuclei_channel_threshold</code> and <code>erosion_factor</code>

In [33]:
# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3
marker_channel = 1

# Define the intensity threshold above which a cell is considered positive for a marker
marker_channel_threshold = 40

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
erosion_factor = 3

# Slice the nuclei and marker stack
nuclei_img = img_mip[nuclei_channel, :, :]
marker_img = img_mip[marker_channel, :, :]

<h3>Mask the input image with the user defined labels and extract data</h3>

In [34]:
for label_name, label_array in zip(layer_names, layer_labels):

    # Perform maximum intensity projection (MIP) from the label stack
    label_mip = np.max(label_array, axis=0)

    # We will create a mask where label_mip is greater than or equal to 1
    mask = label_mip >= 1

    # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
    masked_nuclei_img = np.where(mask, nuclei_img, 0)
    masked_marker_img = np.where(mask, marker_img, 0)
    viewer.add_image(masked_nuclei_img, name=f"{label_name}_nuclei")
    viewer.add_image(masked_marker_img, name=f"{label_name}_marker")

    # Segment nuclei and return labels
    nuclei_labels = segment_nuclei_2d(masked_nuclei_img)
    viewer.add_labels(nuclei_labels, name=f"{label_name}_nuclei_labels")
    # Select marker positive nuclei
    marker_mip, processed_region_labels = segment_marker_positive_nuclei (nuclei_labels, masked_marker_img, marker_channel_threshold, erosion_factor)
    viewer.add_labels(processed_region_labels, name=f"{label_name}_marker+_nuclei")

    # Extract your information of interest
    total_nuclei = len(np.unique(nuclei_labels)) - 1
    marker_pos_nuclei = len(np.unique(processed_region_labels)) - 1

    # Create a dictionary containing all extracted info per masked image
    stats_dict = {
                "filename": filename,
                "ROI": label_name,
                "total_nuclei": total_nuclei,
                "marker+_nuclei": marker_pos_nuclei,
                "%_marker+_cells": (marker_pos_nuclei * 100) / total_nuclei,
                "nuclei_ch": nuclei_channel,
                "marker_ch": marker_channel,
                "marker_int_threshold": marker_channel_threshold,
                "erosion_factor": erosion_factor
                }

    # Append the current data point to the stats_list
    stats.append(stats_dict)    

<h3>Data saving</h3>


In [35]:
# Define output folder for results
results_folder = "./results/"

# Create the necessary folder structure if it does not exist
try:
    os.mkdir(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# Transform into a dataframe to store it as .csv later
df = pd.DataFrame(stats)

# Overwrite the .csv with new data points each round
df.to_csv("./results/marker_+_label_2D.csv", index=True)

df

Output folder already exists: ./results/


,filename,ROI,total_nuclei,marker+_nuclei,%_marker+_cells,nuclei_ch,marker_ch,marker_int_threshold,erosion_factor
0,HI 1 Ipsilateral Mouse 8 slide 6 Neun Red Ca...,full_image,5172,2154,41.647332,3,1,40,3
